In [2]:
from sympy import *
import numpy as np
import plotly.graph_objects as go
from IPython.display import display, Math
from scipy.integrate import odeint

Введите свою функцию $W_р$(s) разомкнутой системы:

In [3]:
# Define open-loop transfer function W_open(s)
s = Symbol('s')

#↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
W_open = (0.3*s+1) / ((0.25*s+1)*(0.04*s**2+0.07*s+1))
#↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑

# Display open-loop transfer function W_open(s)
display(Math('W_р(s) = ' + latex(W_open)))

# Print as LaTeX code for copy-paste to LaTeX document
print('Скопируйте код ниже и вставьте в свой LaTeX-файл:')
print('\\begin{equation}' + '\n' + '\t' + 'W_р(s) = ' +  latex(W_open) + '\n' + '\end{equation}')

<IPython.core.display.Math object>

Скопируйте код ниже и вставьте в свой LaTeX-файл:
\begin{equation}
	W_р(s) = \frac{0.3 s + 1}{\left(0.25 s + 1\right) \left(0.04 s^{2} + 0.07 s + 1\right)}
\end{equation}


Функция $W_з$(s) замкнутой системы:

In [4]:
# Define closed-loop transfer function W_closed(s)
Y_closed = (fraction(W_open)[0]).simplify()
U_closed = (fraction(W_open)[1] + fraction(W_open)[0]).simplify()
a = U_closed.coeff(s, degree(U_closed, s))
Y_closed = (Y_closed / a).simplify()
U_closed = (U_closed / a).simplify()
W_closed = Y_closed / U_closed

# Display closed-loop transfer function W_closed(s)
display(Math('W_з(s) = ' + latex(W_closed)))

# Print as LaTeX code for copy-paste to LaTeX document
print('Скопируйте код ниже и вставьте в свой LaTeX-файл:')
print('\\begin{equation}' + '\n' + '\t' + 'W_з(s) = ' +  latex(W_closed.simplify()) + '\n' + '\end{equation}')

<IPython.core.display.Math object>

Скопируйте код ниже и вставьте в свой LaTeX-файл:
\begin{equation}
	W_з(s) = \frac{30.0 s + 100.0}{1.0 s^{3} + 5.75 s^{2} + 62.0 s + 200.0}
\end{equation}


Шаг по времени для моделирования:

In [5]:
Y_open = fraction(W_open)[0]
U_open = fraction(W_open)[1]
factors = factor_list(Y_open)[1] + factor_list(U_open)[1]
# Min of time constants
tau = min([pow(factor[0].coeff(s, degree(factor[0], s)), 1/degree(factor[0], s)) for factor in factors])
# Display time constant
display(Math('tau = ' + latex(tau)))

<IPython.core.display.Math object>

Составление системы дифференциальных уравнений из передаточной функции

In [6]:
# Define function y(t) and function u(t)
t = Symbol('t')
y = Function('y')(t)
u = Function('u')(t)

# Define system of differential equations from transfer function
def system_of_diff_eqs(W):
    # Define variables
    variables = []
    # Find Y(s) and U(s) from transfer function W(s)
    Y = fraction(W)[0].expand()
    U = fraction(W)[1].expand()
    # Define system of differential equations
    eqs = []
    # Define the order of U(s)
    n = degree(U, s)
    # Define x_1(t) = y(t)
    x_1 = Function('x_1')(t)
    # Add equations to the system
    for i in range(1, n):
        # Define x_i(t)
        x_i = Function('x_' + str(i))(t)
        # Define x_{i+1}(t)
        x_i_next = Function('x_' + str(i+1))(t)
        # Append new variable to the list of variables
        variables.append(x_i)
        # Append new equation to the system
        eqs.append(Eq(diff(x_i, t), -U.coeff(s, n-i)*x_1 + x_i_next + Y.coeff(s, n-i)*u)) 
    # Define x_n(t)
    x_n = Function('x_' + str(n))(t)
    # Append x_n(t) to the list of variables
    variables.append(x_n)
    # Append new equation to the system
    eqs.append(Eq(diff(x_n, t), -U.coeff(s, 0)*x_1 + Y.coeff(s, 0)*u))
    # Append last equation to the system
    eqs.append(Eq(y, x_1))
    # Return system of differential equations and list of variables
    return eqs, variables

In [7]:
# Find system of differential equations from transfer function W_closed(s)
eqs, variables = system_of_diff_eqs(W_closed)

# Display system of differential equations
latex_str = '\\begin{cases}'
for i in range(len(eqs)):
    latex_str += latex(eqs[i]) + '\\\\'
latex_str += '\\end{cases}'
display(Math(latex_str))

# Print latex code of system of differential equations
print('Скопируйте код ниже и вставьте в свой LaTeX-файл:')
print('\\begin{cases}')
for i in range(len(eqs)-1):
    print('\t'+latex(eqs[i]) + '\\\\')
print('\t'+latex(eqs[-1]))
print('\\end{cases}')

<IPython.core.display.Math object>

Скопируйте код ниже и вставьте в свой LaTeX-файл:
\begin{cases}
	\frac{d}{d t} \operatorname{x_{1}}{\left(t \right)} = - 5.75 \operatorname{x_{1}}{\left(t \right)} + \operatorname{x_{2}}{\left(t \right)}\\
	\frac{d}{d t} \operatorname{x_{2}}{\left(t \right)} = 30.0 u{\left(t \right)} - 62.0 \operatorname{x_{1}}{\left(t \right)} + \operatorname{x_{3}}{\left(t \right)}\\
	\frac{d}{d t} \operatorname{x_{3}}{\left(t \right)} = 100.0 u{\left(t \right)} - 200.0 \operatorname{x_{1}}{\left(t \right)}\\
	y{\left(t \right)} = \operatorname{x_{1}}{\left(t \right)}
\end{cases}


Решение системы дифференциальных уравнений для передаточной функции замкнутой системы:

In [9]:
# Solve system of differential equations eqs
# Initial conditions:
# u(t=0) = Heaviside(t)
# x_i(t=0) = 0

# Define function for solving system of differential equations
def f(y, t):
    # Define dictionary of variables
    variables_dict = {}
    # Fill dictionary of variables
    for i in range(len(variables)):
        variables_dict[variables[i]] = y[i]
    # Define list of derivatives
    dydt = []
    # Fill list of derivatives
    for i in range(len(eqs)-1):
        dydt.append(eqs[i].subs(variables_dict).subs(u, u0(t)).rhs)
    # Return list of derivatives
    return dydt

# Define function for solving system of differential equations
def solve_system_of_diff_eqs(eqs, variables, ts, u0, f):
    # Define initial conditions
    y0 = [0] * len(variables)
    # Solve system of differential equations
    y = odeint(f, y0, ts)
    # Return list of solutions
    return y

# Find solution
time = 8
if float(tau) > 0.01:
    tau = 0.01
ts = np.arange(0, time, float(tau))
u0 = lambda t: 1 if t >= 0 else 0
solution = solve_system_of_diff_eqs(eqs, variables, ts, u0, f)

KeyboardInterrupt: 

In [ ]:
# Plot solution using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=ts, y=solution[:, 0], name='y(t)', line_shape='spline', line=dict(color='blue', width=3)))
fig.add_trace(go.Scatter(x=ts, y=[u0(t) for t in ts], name='u(t)', line_shape='spline', line=dict(color='red', width=3)))
fig.update_layout(xaxis_title='t', yaxis_title='y(t), u(t)')
fig.update_layout(template='plotly_white')
fig.update_layout(font=dict(size=20))
fig.update_layout(width=680, height=420)
fig.write_image('solution_first.pdf')
fig.show()

Система дифференциальных уравнений при добавлении в схему ключа:

In [ ]:
# Display system of differential equations
latex_str = '\\begin{cases}'
for i in range(len(eqs)):
    if i == len(eqs)-2:
        u_star = Function('u^*')(t)
        x_1_star = Function('x_1^*')(t)
        latex_str += latex(eqs[i].subs(u, u_star).subs(variables[0], x_1_star)) + '\\\\'
    else:
        latex_str += latex(eqs[i]) + '\\\\'
latex_str += '\\end{cases}'
display(Math(latex_str))

# Print latex code of system of differential equations
print('Скопируйте код ниже и вставьте в свой LaTeX-файл:')
print('\\begin{cases}')
for i in range(len(eqs)-1):
    if i == len(eqs)-2:
        u_star = Function('u^*')(t)
        x_1_star = Function('x_1^*')(t)
        print('\t' + latex(eqs[i].subs(u, u_star).subs(variables[0], x_1_star)) + '\\\\')
    else:
        print('\t' + latex(eqs[i]) + '\\\\')
print('\t'+latex(eqs[-1]))
print('\\end{cases}')

<IPython.core.display.Math object>

Скопируйте код ниже и вставьте в свой LaTeX-файл:
\begin{cases}
	\frac{d}{d t} \operatorname{x_{1}}{\left(t \right)} = - 5.75 \operatorname{x_{1}}{\left(t \right)} + \operatorname{x_{2}}{\left(t \right)}\\
	\frac{d}{d t} \operatorname{x_{2}}{\left(t \right)} = 30.0 u{\left(t \right)} - 62.0 \operatorname{x_{1}}{\left(t \right)} + \operatorname{x_{3}}{\left(t \right)}\\
	\frac{d}{d t} \operatorname{x_{3}}{\left(t \right)} = 100.0 \operatorname{u^{*}}{\left(t \right)} - 200.0 \operatorname{x^{*}_{1}}{\left(t \right)}\\
	y{\left(t \right)} = \operatorname{x_{1}}{\left(t \right)}
\end{cases}


In [ ]:
# Define quantization step
T = 0.05

# Define key function
def key_function(x, t, T):
    # Define key function
    if abs(round(t/T)*T - t) < tau/2:
        return x/tau
    else:
        return 0

# Define function for solving system of differential equations
def f2(y, t):
    # Define dictionary of variables
    variables_dict = {}
    # Fill dictionary of variables
    for i in range(len(variables)):
        variables_dict[variables[i]] = y[i]
    # Define dictionary of variables for key function
    variables_dict2 = {}
    # Fill dictionary of variables for key function
    for i in range(len(variables)):
        variables_dict2[variables[i]] = key_function(y[i], t, T)
    # Define list of derivatives
    dydt = []
    # Fill list of derivatives
    for i in range(len(eqs)-1):
        if i == len(eqs)-2:
            dydt.append(eqs[i].subs(variables_dict2).subs(u, key_function(u0(t), t, T)).rhs)
        else:
            dydt.append(eqs[i].subs(variables_dict).subs(u, u0(t)).rhs)
    # Return list of derivatives
    return dydt

# Find solution
time = 8
if float(tau) > 0.01:
    tau = 0.01
ts = np.arange(0, time, float(tau))
u0 = lambda t: 1 if t >= 0 else 0
solution = solve_system_of_diff_eqs(eqs, variables, ts, u0, f2)

In [ ]:
# Plot solution using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=ts, y=solution[:, 0], name='y(t)', line_shape='spline', line=dict(color='blue', width=3)))
fig.add_trace(go.Scatter(x=ts, y=[u0(t) for t in ts], name='u(t)', line_shape='spline', line=dict(color='red', width=3)))
fig.update_layout(xaxis_title='t', yaxis_title='y(t), u(t)')
fig.update_layout(template='plotly_white')
fig.update_layout(font=dict(size=20))
fig.update_layout(width=680, height=420)
fig.write_image('solution_second.pdf')
fig.show()